# Building an ensemble of future baseline scenarios
The scenario functions enable users to quickly generate scenarios by reassembling years from the historical data. Reusing past years can offer several advantages:
1. It is easier than creating boundary stresses from downscaled climate scenarios.
2. It is likely more accurate than ad-hoc models that may not accurately 
   capture the complex relationship between weather and groundwater stresses.
3. Past years are familiar and easier to communicate to stakeholders. For example: 
   "here is a scenario in which a water year like 2013 occurs three years in a row".
4. It is easy to generate an ensemble of scenarios.

This notebook will demonstrate how one can quickly and easily generate an ensemble
of scenarios that can be used as a baseline for a water management scenario. 

In [25]:
# IMPORT libraries
import os
import flopy
from mfmodify import get_sp_data, scenario_from_repeat_years
from mfmodify.utils import get_idomain_df, annual_summary_from_gwf

Point to the location of the existing historical model and the directory where we'll save scenarios.

In [11]:
# INPUT
sim_ws_orig = os.path.join('historic_models', 'model_v1-metric')
scenario_dir = os.path.join('scenario_tests', 'tvgwfm')
n_years = 10

Create a few functions 

In [32]:
def get_spaced_cells(node_cells, ncells):
    total_cells = len(node_cells)
    step = total_cells / ncells
    indices = np.round(np.arange(step/2, total_cells, step)).astype('int')
    spaced_cells = (
        node_cells
        .iloc[indices, :]
        .cellid.tolist()
    )
    return spaced_cells

First we'll create a simple scenario by rerunning the final years of the historic period, using the final stress period of the historic solution as the initial condition. We'll add a few head observations spread around the model for comparisons. We'll also remove the oc file so the hds and bud files don't save.

In [5]:
# load the original gwf model to get the simulation years and to place hdobs
# get sim and gwf objects
sim_orig = flopy.mf6.MFSimulation.load(sim_ws=sim_ws_orig, verbosity_level=0)
gwf_orig = sim_orig.get_model()

In [ ]:
# get an annual summary of the model years from the original model lst file
add_sum_orig = annual_summary_from_gwf(gwf_orig)
add_sum_orig.tail(n_years)

,total_recharge,total_discharge,net_storage
year,,,
2006,61121836.0,62957696.0,-1835857.50
2007,60412532.0,65279984.0,-4867452.00
2008,61004020.0,61700516.0,-696494.25
2009,64962272.0,61261284.0,3700987.50
2010,60769560.0,61920100.0,-1150543.25
2011,62719968.0,61779636.0,940331.00
2012,62824416.0,63705264.0,-880846.75
2013,59340544.0,68316424.0,-8975886.00
2014,65128788.0,66681316.0,-1552525.00


In [17]:
# get the model years and define the initial condition month and year
scen_years_base = add_sum_orig.index[-n_years:].values.tolist()
ic_mon_year = (12, scen_years_base[-1])
# set scenario directory name - calling this scenario 'base'
sim_ws_scenbase = os.path.join(scenario_dir, f'base-last_{n_years}_years')

We need the hds file to get the initial condition for the scenario. The .gitignore 
file ignores the large hds and bud files, so we need to run the original model. This 
example takes 2.5 minutes on my laptop.

In [19]:
sim_orig.run_simulation(silent=True)

(True, [])

Now we'll create the new simulation object, but we won't create files or run until 
after we add a hds and remove the oc file.

In [ ]:
# make a scenario using the scenario builder function
sim_scen_base = scenario_from_repeat_years(
    sim_ws_orig, # original scenario directory
    scen_years_base, # the scenario years to assemble
    ic_mon_year, # year and month from original to use as initial conditions
    new_sim_ws=sim_ws_scenbase # new model file directory
)

Loading simulation from historic_models\model_v1-metric
generating new simulation with modified sp and timeseries info


In [24]:
# get gwf object
gwf_scen_base = sim_scen_base.get_model()
# remove oc
oc = gwf_scen_base.get_package('oc').remove()

For head observations, we'll just choose some equally spaced cells around the model

In [ ]:
# get idomain locations as a dataframe (using helper function)
idomain_df = get_idomain_df(gwf_scen_base)
# get only active cells as options, let's stick to layer 1
cells_df = idomain_df.query('idomain==1').query('layer==1')
# choose evenly spaced cells (using helper function)
obs_cells = get_spaced_cells(cells_df, 6)

,nodeid,cellid,idomain,layer,row,column
0,0,"(0, 0, 0)",0,0,0,0
1,1,"(0, 0, 1)",0,0,0,1
2,2,"(0, 0, 2)",0,0,0,2
3,3,"(0, 0, 3)",0,0,0,3
4,4,"(0, 0, 4)",0,0,0,4
